In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import model_from_json
from datetime import timedelta

import statsmodels.formula.api as smf

from src.dynamic_pricing_data_loader import cargar_y_preparar_datos

from Tools.ExtractData import GetDataFromETL
from Tools.Tools4TrainRN import RedNeuronal
from Tools.Tools4ProofRN import ProofRedNeuronal
from Tools.Tools4Elasticity import Elasticity
from Tools.Tools4Clasify import K_means
from Tools.Tools4ClasSuper import ClusteringSupervisado
from Tools.Tools4ProofCluster import PredictorClusters


In [2]:
Data= GetDataFromETL(True)

El archivo JSON ha sido actualizado con las nuevas rutas.
Memoria usada antes: 5201.17 MB
Memoria usada después: 1001.59 MB
Reducción: 80.7%


In [3]:
#Elasticidad
Df_Elas= Data.D4_Elasticity() # Extraccion de los datos para elasticidad

Elas4OriDest= Elasticity(Df_Elas).Df

In [4]:
Elas4OriDest.to_csv('Elasticidades.csv')

In [5]:
# 1. Asegurar formato de fecha en la columna original
Elas4OriDest['FECHA_CORRIDA'] = pd.to_datetime(Elas4OriDest['FECHA_CORRIDA'], dayfirst=True)

# 2. Agrupar por Origen-Destino, Mes y Día
# Usamos nombres de columna que pd.to_datetime reconoce (year, month, day)
resumen = (Elas4OriDest
           .groupby(['ORIGEN_DESTINO', 
                     Elas4OriDest['FECHA_CORRIDA'].dt.month.rename('month'), 
                     Elas4OriDest['FECHA_CORRIDA'].dt.day.rename('day')])['ELASTICIDADES']
           .mean()
           .reset_index())

# 3. Crear la columna de año para el ensamble
resumen['year'] = Elas4OriDest['FECHA_CORRIDA'].dt.year.max()+1

# 4. Crear la FECHA_NUEVA manejando el error del 29 de febrero
# errors='coerce' pondrá NaT en el 29 de febrero de 2026
resumen['FECHA_NUEVA'] = pd.to_datetime(resumen[['year', 'month', 'day']], errors='coerce')

# 5. Manejo del 29 de febrero (Opcional):
# Si quieres eliminar ese día que no existe en 2026:
resumen = resumen.dropna(subset=['FECHA_NUEVA'])

# 6. Limpieza final de columnas
resultado_final = resumen[['ORIGEN_DESTINO', 'FECHA_NUEVA', 'ELASTICIDADES']]
resultado_final.columns = ['ORIGEN_DESTINO', 'FECHA_CORRIDA', 'ELASTICIDADES']

# Ordenar por destino y fecha
resultado_final = resultado_final.sort_values(['ORIGEN_DESTINO', 'FECHA_CORRIDA'])


In [6]:
resultado_final.to_excel('Elas_2026.xlsx')